# BAMBOO: Binary descriptor based on AsymMetric pairwise BOOsting

In this notebook we include the implementation of the BAMBOO descriptor to provide a compressed representation of probe requests.

## Libraries and Configurations

Import configuration files

In [13]:
from configparser import ConfigParser

config = ConfigParser()
config.read("../config.ini")

['../config.ini']

Import **data libraries**

In [14]:
import pandas as pd

Import **other libraries**

In [15]:
from rich.progress import Progress
from rich import traceback

traceback.install()

from tqdm.notebook import tqdm

In [16]:
import numpy as np

## Import Data

## BAMBOO

Input:
- Ground truth relationships $\langle x_{a(n)}, x_{b(n)}; y_n\rangle$
  - $n=1,..,N$
  - $y_n \in \{+1, -1\}$
- A set of filters $\mathcal{H} = \{h_1 , ..., h_F\}$
- A set of binarization thresholds $\mathcal{T} = \{t_1 , ..., t_T\}$

Output:
- A set of $M<F$ filters $[h_{i(1)}, ..., h_{i(M)}]$
- Corresponding set of binarization thresholds $[t_{j(1)}, ..., t_{j(M)}]$

In [3]:
import random


def generate_random_binary_strings(N, L):
    binary_strings = []
    for _ in range(N):
        binary_string = "".join(random.choice("01") for _ in range(L))
        binary_strings.append(binary_string)
    return binary_strings


# Example usage:
N = 10
L = 24
random_binary_strings = generate_random_binary_strings(N, L)
print(random_binary_strings)

['001000110011101100111001', '100101110111011101010000', '100101000100100101010101', '100000111001010111011001', '101001010100010100001100', '000111001010111101001000', '100001010111100100100111', '101000110100011101100100', '000111101011111101001001', '001010110001000111110101']


In [4]:
data = random_binary_strings

In [8]:
def generate_bitmaps(total_length: int, width: int) -> list:
    """
    Generate bitmaps of 8 consecutive 1s within a longer string of 0s.

    Parameters:
        total_length (int): The total length of the combined bitmaps.

    Returns:
        list: A list containing the bitmaps.
    """
    if total_length % width != 0:
        raise ValueError("Total length must be a multiple of 8.")

    num_bitmaps = total_length // width
    bitmaps = [
        "0" * (i * width) + "1" * width + "0" * ((num_bitmaps - i - 1) * width)
        for i in range(num_bitmaps)
    ]
    return bitmaps

In [10]:
# Example usage:
total_length = 24
filters = generate_bitmaps(total_length, 4)
for filter in filters:
    print(filter)

111100000000000000000000
000011110000000000000000
000000001111000000000000
000000000000111100000000
000000000000000011110000
000000000000000000001111


In [11]:
filters

['111100000000000000000000',
 '000011110000000000000000',
 '000000001111000000000000',
 '000000000000111100000000',
 '000000000000000011110000',
 '000000000000000000001111']

In [12]:
def generate_thresholds(bitmasks):
    """
    Generate thresholds for each bitmask in a set.

    Parameters:
        bitmasks (set): A set containing the bitmasks.

    Returns:
        dict: A dictionary where keys are bitmasks and values are sets of thresholds.
    """
    thresholds_dict = {}
    for bitmask in bitmasks:
        max_ones = bitmask.count("1")
        thresholds = set(range(max_ones + 1))
        thresholds_dict[bitmask] = thresholds
    return thresholds_dict

In [13]:
# # Example usage:
# thresholds_dict = generate_thresholds(filters)
# for bitmask, thresholds in thresholds_dict.items():
#     print(f"Bitmask: {bitmask}, Thresholds: {thresholds}")

Bitmask: 111100000000000000000000, Thresholds: {0, 1, 2, 3, 4}
Bitmask: 000011110000000000000000, Thresholds: {0, 1, 2, 3, 4}
Bitmask: 000000001111000000000000, Thresholds: {0, 1, 2, 3, 4}
Bitmask: 000000000000111100000000, Thresholds: {0, 1, 2, 3, 4}
Bitmask: 000000000000000011110000, Thresholds: {0, 1, 2, 3, 4}
Bitmask: 000000000000000000001111, Thresholds: {0, 1, 2, 3, 4}


In [14]:
thresholds = [0, 1, 2, 3, 4]

In [ ]:
LABELS = {}